In [1]:
# This file aims at converting units that are not grammes in grammes
# we consider here that : 1 c.à.c = 5 g and 1 c.à.s = 15 grammes

In [1]:
!pip install lxml 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 18.7 MB/s eta 0:00:0000:0100:01


In [2]:
import requests
from bs4 import BeautifulSoup
import html.parser
import re
import numpy as np
from tqdm.auto import tqdm
import pandas as pd


In [4]:
corres = open("references/Poids.txt", "r")
poids=corres.readlines()
dicopoids={}
for ligne in poids:
    dicopoids[str(''.join(z for z in ligne if not z.isdigit()).replace(" \n", "").lower())]=int("".join(re.findall('\d',str(ligne))))
     

In [5]:
corres2 = open("references/Quantificateurs.txt", "r")
quantif=corres2.readlines()
dicoquantif={}
for ligne in quantif:
    dicoquantif[str(''.join(z for z in ligne if not z.isdigit()).replace(" \n", "").lower())]=int("".join(re.findall('\d',str(ligne))))
     

In [6]:
def keskimank(dico):
    for x,y in dico.items():
                for k,v in y['recette'].items():
                    l=str(''.join(z for z in v if not z.isdigit()))
                    if l not in ['.g','g']:

                        print(k,v)

                    

In [7]:
def convert(dico):
    for x,y in dico.items():
        
            
                for k,v in y['recette'].items():
                    v=str(v)
                    if v != '':
                        if '⁄' in v:
                            if '1⁄4' in v:
                                n=0.25
                            if '1⁄3' in v:
                                n=0.33
                            if '1⁄2' in v:
                                n=0.5
                            for a,b in dicopoids.items():
                                if a in k:
                                    y['recette'].update({k:str(b*n)+"g"})
                            for a,b in dicoquantif.items():
                                if l in ['.'+a,a,a+'s','.'+a+'s','⁄'+a,'⁄'+a+'s'] :
                                    y['recette'].update({k:str(b*n)+"g"})       
                            
                
                        else:
                            n=float(("".join(re.findall('\d',str(v)))))
                
                            l=str(''.join(z for z in v if not z.isdigit()))
                
                            if l not in ['g','.g']:    
                                for a,b in dicopoids.items():
                                    if a in k:
                                        y['recette'].update({k:str(b*n)+"g"})
                                for a,b in dicoquantif.items():
                                    if l in ['.'+a,a,a+'s','.'+a+'s'] :
                                        y['recette'].update({k:str(b*n)+"g"})       
                                    
                    elif v=="": #on définit une valeur standardisé pour les éléments manquants. Ces élements sont souvent utilisés en des quantités limitées.
                        y['recette'].update({k:str(2)+"g"})

In [8]:
def find_all_dishes(search : str, nb_of_recipes : int): 
#returns a list with all the url of the recipes corresponding to the research
    
    #making the research compatible with an url  : 
    key_word = search.lower().replace(' ', '-')
    first_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=" + key_word
    
    #going to the first proposals (i.e. the first proposal page) for key_word : 
    current_page = BeautifulSoup(requests.get(first_url).text, features="html.parser") 
    
    #1 : getting all the urls for the proposal pages : 
    list_url = [first_url]
    for url in list_url :
        current_page = BeautifulSoup(requests.get(url).text, features="html.parser")
        found_pages = current_page.findAll('a', {'class' : 'SHRD__sc-1ymbfjb-1 MTkAM'})
        #all the links that appear on the current page to go to other pages 
        for page in found_pages : 
            new_url = "https://www.marmiton.org" + page.get('href')
            if new_url not in list_url : 
                list_url.append(new_url)
    
    #2 : getting all the urls for the recipe pages : 
    list_dishes_url = []
    for url in list_url : 
        current_page = BeautifulSoup(requests.get(url).text, features="html.parser")
        found_dishes = current_page.findAll('a', {'class' : 'MRTN__sc-1gofnyi-2 gACiYG'})
        #all the recipes found on the current page
        for dish in found_dishes:
            href = dish.get('href')
            if href[0:17] == '/recettes/recette' : 
            #sometimes, marmiton proposes albums, or videos without recipes instead of recipes
                new_url = "https://www.marmiton.org" + href
                if len(list_dishes_url) < nb_of_recipes:
                    list_dishes_url.append(new_url)
                
    return list_dishes_url
    
    
def find_recipe(dish_url : str): 
#returns a list corresponding to the url containing : 
#1) the name of the recipe 
#2) a dictionnary with the recipe 
    
    soup = BeautifulSoup(requests.get(dish_url).text,features="html.parser")
    
    #0 : for the title of the recipe : 
    recipe_title = soup.find('h1', {'class' : 'SHRD__sc-10plygc-0 itJBWW'})
    recipe_title = recipe_title.get_text(separator="") 
    
    
    #1 : for the ingredients
    
    ingredients_cells = soup.findAll('div', {'class' : 'MuiGrid-root MuiGrid-item MuiGrid-grid-xs-3 MuiGrid-grid-sm-3'})
    ingredients_names = []
    for cell in ingredients_cells:
        ingredient = cell.findAll('span', {'class' : 'RCP__sc-8cqrvd-3 itCXhd'})
        if len(ingredient) == 0:
            ingredient = cell.findAll('span', {'class' : 'RCP__sc-8cqrvd-3 cDbUWZ'})
        ingredients_names.append(ingredient[0].string)
    
    #2: for the quantites
    qte = soup.findAll('span', {'class' : 'SHRD__sc-10plygc-0 epviYI'})
    qtes = []
    for q in qte:
        string = q.text.replace("\xa0", "")
        qtes.append(string)
        
    #3: creating a dictionnary for the recipe : 
    recipe = {}
    for i in range (len(ingredients_names)) :
        recipe[ingredients_names[i]] = qtes[i]
    
    #4 : for the number of people : 
    nb_people = soup.find('span', {'class' : 'SHRD__sc-w4kph7-4 knYsyq'}) or soup.find('span', {'class' : 'SHRD__sc-w4kph7-4 hYSrSW'}) or '4' 
    #by default, we say that the recipe is for 4 persons 
    if nb_people != '4' : 
        nb_people = nb_people.get_text(separator="")
    nb_people = float(nb_people)
    
    return recipe_title, recipe, nb_people
            
def find_all_recipes(search : str, nb_of_recipes : int) : 
#returns a dataframe with all the the recipes corresponding to the research
    
    list_dishes_url = find_all_dishes(search, nb_of_recipes)
    all_recipes = {}
    for dish in tqdm(list_dishes_url) : 
        all_recipes[find_recipe(dish)[0]] = {'lien' : dish,'recette' : find_recipe(dish)[1], 'nombre de personnes' : find_recipe(dish)[2]}
        #we should add other features like the mark, the nb of people, the number of opinions...
        
    recipes_names = list(all_recipes.keys())
    convert(all_recipes)
    keskimank(all_recipes)
    print(all_recipes)
    
    
    return 
        
                
    

In [9]:

find_all_recipes('ile flottante',20)



  0%|          | 0/20 [00:00<?, ?it/s]

lait 3⁄4l
{'Ile flottante à la crème anglaise au micro-ondes': {'lien': 'https://www.marmiton.org/recettes/recette_ile-flottante-a-la-creme-anglaise-au-micro-ondes_85957.aspx', 'recette': {'sucre': '100g', 'sel': '5.0g', 'maïzena': '5.0g', 'lait': '500.0g', 'vanille': '10.0g', 'oeufs': '330.0g'}, 'nombre de personnes': 6.0}, 'Ile flottante': {'lien': 'https://www.marmiton.org/recettes/recette_ile-flottante_21741.aspx', 'recette': {'lait': '600.0g', 'sucre en poudre': '60g', 'vanille': '10.0g', 'sel': '5.0g', 'sucre glace': '130g', 'amandes effilées': '60g', 'oeufs': '275.0g'}, 'nombre de personnes': 4.0}, 'Ile flottante aux fruits rouges (allégée)': {'lien': 'https://www.marmiton.org/recettes/recette_ile-flottante-aux-fruits-rouges-allegee_30911.aspx', 'recette': {'fromage blanc': '200g', 'sucre glace': '20g', 'fruits rouges': '200g', 'édulcorant': '30.0g', "blanc d'oeuf": '55.0g'}, 'nombre de personnes': 2.0}, 'Pommes sur île flottante': {'lien': 'https://www.marmiton.org/recettes/rec

In [10]:
test

['Tourte au poulet, champignons et lardons',
 'Tourte Poireaux, Jambon et Chèvre',
 'La tourte aux champignons, si simple et si bon!',
 'Tourte courgette et feta',
 'Tourte au Poulet',
 'Tourte au thon',
 'Tourte à la parisienne',
 "Pâte à tarte ou tourte à l'huile d'olive",
 'Tourte aux Myrtilles',
 'Tourte canadienne aux pommes',
 'Tourte facile chèvre et menthe',
 'Tourte de Noël végétarienne de Coralie',
 'TOURTE AUX POMMES DE TERRE',
 'Tourte aux poireaux, lardons et reblochon',
 'Tourte au confit de canard']

In [7]:
def convert(dico):
    for x,y in dico.items():
        
            
                for k,v in y['recette'].items():
            
                    if v != '':
                        if '⁄' in v:
                            if '1⁄4' in v:
                                n=0.25
                            if '1⁄3' in v:
                                n=0.33
                            if '1⁄2' in v:
                                n=0.5
                            for a,b in dicopoids.items():
                                if a in k:
                                    y['recette'].update({k:str(b*n)+"g"})
                            for a,b in dicoquantif.items():
                                if a in v:
                                    y['recette'].update({k:str(b*n)+"g"})       
                            
                
                        else:
                            n=float(("".join(re.findall('\d',str(v)))))
                
                            l=str(''.join(z for z in v if not z.isdigit()))
                
                            if l not in ['g','.g']:    
                                for a,b in dicopoids.items():
                                    if a in k:
                                        y['recette'].update({k:str(b*n)+"g"})
                                for a,b in dicoquantif.items():
                                    if a in v:
                                        y['recette'].update({k:str(b*n)+"g"})       
                                        
                    elif v=="": #on définit une valeur standardisé pour les éléments manquants. Ces élements sont souvent utilisés en des quantités limitées.
                        y['recette'].update({k:str(2)+"g"})
                
    
                    
                    
            
            
                
            
            

In [16]:
convert(test)

KeyError: 'recette'

In [75]:
test

{"Tarte aux pommes à l'Alsacienne": {'lien': 'https://www.marmiton.org/recettes/recette_tarte-aux-pommes-a-l-alsacienne_11457.aspx',
  'recette': {'pommes': '525.0g',
   'crème fraîche': '25000.0g',
   'sucre en poudre': '100g',
   'sucre vanillé': '15.0g',
   'pâte brisée': '1',
   'oeufs': '110.0g'},
  'nombre de personnes': 6.0},
 'Tarte thon, tomate et moutarde': {'lien': 'https://www.marmiton.org/recettes/recette_tarte-thon-tomate-et-moutarde_17226.aspx',
  'recette': {'tomates': '250.0g',
   'moutarde': '45.0g',
   'crème fraîche': '20000.0g',
   'lait': '45.0g',
   'sel': '5.0g',
   'poivre': '10.0g',
   'pâte feuilletée': '1.0g',
   'thon': '500.0g',
   'oeufs': '110.0g'},
  'nombre de personnes': 4.0},
 'Tarte au citron meringuée': {'lien': 'https://www.marmiton.org/recettes/recette_the-tarte-au-citron-meringuee_22082.aspx',
  'recette': {'sucre glace': '100g',
   'levure chimique': '2.5g',
   "blancs d'oeuf": '110.0g',
   'citrons': '440.0g',
   'sucre semoule': '70g',
   'ma

In [90]:
keskimank(test)

chèvre frais 2
gruyère râpé 1bonne poignée
escalopes de poulet 3
reblochon 1


In [28]:
def bouchetrou(dico):
    for x,y in dico.items():
                element=[]
                for k,v in y['recette'].items():
                    l=str(''.join(z for z in v if not z.isdigit()))
                    if l not in ['.g','g']:
                        element=element+[k]
                        
                for a in element:
                    del y['recette'][a]


In [30]:
bouchetrou(test)
keskimank(test)
test

{'Truites simplissimes en papillottes': {'lien': 'https://www.marmiton.org/recettes/recette_truites-simplissimes-en-papillottes_69681.aspx',
  'recette': {'persil': '50.0g',
   'ail': '20.0g',
   "huile d'olive": '2400.0g',
   'citron': '2g',
   'poivre': '2g',
   'sel': '2g',
   'truites': '900000.0g'},
  'nombre de personnes': 4.0},
 'Papillotes de truite au barbecue': {'lien': 'https://www.marmiton.org/recettes/recette_papillotes-de-truite-au-barbecue_17893.aspx',
  'recette': {'ail': '2g',
   'basilic': '2g',
   'oignon': '2g',
   "huile d'olive": '2g',
   'truites': '900000.0g'},
  'nombre de personnes': 4.0},
 'Filet de truite aux amandes (four)': {'lien': 'https://www.marmiton.org/recettes/recette_filet-de-truite-aux-amandes-four_68018.aspx',
  'recette': {'amandes effilées': '80g',
   'ail': '2g',
   "huile d'olive": '2g',
   'truite': '3000000.0g',
   'crème liquide': '20000.0g'},
  'nombre de personnes': 2.0},
 'Filet de truite sauce citronnée (très facile)': {'lien': 'https:

In [50]:
def usable(dico):
    for x,y in dico.items():
        for k,v in y['recette'].items():
                n=float(v.replace('g',''))
                y['recette'].update({k:float(n)})

In [51]:
usable(test)
test

{'Steak tartare': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare_18121.aspx',
  'recette': {'moutarde de Dijon': 5.0,
   'oignon': 15.0,
   'câpres': 15.0,
   'sauce worcestershire': 5.0,
   'ketchup': 15.0,
   'tabasco': 2.0,
   'poivre': 2.0,
   'sel': 2.0,
   "huile d'olive": 30.0,
   'persil': 5.0,
   'boeuf': 250.0,
   "jaune d'oeuf": 55.0},
  'nombre de personnes': 2.0},
 'Steak tartare arrangé': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare-arrange_50144.aspx',
  'recette': {'boeuf haché': 450.0,
   'huile': 2.0,
   'moutarde': 2.0,
   'ketchup': 2.0,
   'tabasco': 2.0,
   'poivre': 2.0,
   'sel': 2.0,
   'câpres': 2.0,
   'persil': 2.0,
   'oeuf': 55.0,
   'échalote': 40.0,
   'cornichons': 2.0,
   'ciboulette': 2.0},
  'nombre de personnes': 2.0},
 'Steak tartare végétarien': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare-vegetarien_30154.aspx',
  'recette': {'carotte': 500.0,
   'câpres': 30.0,
   'poivron rouge': 300.

In [52]:
def parpersonne(dico):
    for x,y in dico.items():
        for k,v in y['recette'].items():
                
                y['recette'].update({k:v/y['nombre de personnes']})

In [53]:
parpersonne(test)

In [54]:
test

{'Steak tartare': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare_18121.aspx',
  'recette': {'moutarde de Dijon': 2.5,
   'oignon': 7.5,
   'câpres': 7.5,
   'sauce worcestershire': 2.5,
   'ketchup': 7.5,
   'tabasco': 1.0,
   'poivre': 1.0,
   'sel': 1.0,
   "huile d'olive": 15.0,
   'persil': 2.5,
   'boeuf': 125.0,
   "jaune d'oeuf": 27.5},
  'nombre de personnes': 2.0},
 'Steak tartare arrangé': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare-arrange_50144.aspx',
  'recette': {'boeuf haché': 225.0,
   'huile': 1.0,
   'moutarde': 1.0,
   'ketchup': 1.0,
   'tabasco': 1.0,
   'poivre': 1.0,
   'sel': 1.0,
   'câpres': 1.0,
   'persil': 1.0,
   'oeuf': 27.5,
   'échalote': 20.0,
   'cornichons': 1.0,
   'ciboulette': 1.0},
  'nombre de personnes': 2.0},
 'Steak tartare végétarien': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare-vegetarien_30154.aspx',
  'recette': {'carotte': 125.0,
   'câpres': 7.5,
   'poivron rouge': 75.0,
  

In [55]:
def conversion(dico):
    convert(dico)
    usable(dico)
    parpersonne(dico)